# Modified Net Ratings

In [ ]:
import os, sys

sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath("__file__"))))
from nbafuns import *

fig_DIR = "../figs/teams/"
box_DIR = "../data/box/"
img_DIR_T = "../data/images/teams/"

team_dict, team_list = get_teams()

## Time Decay Net Rating

In [ ]:
df0 = get_box("T","Base",False,[2024])
df0 = df0[["game_id","game_date"]]
df0.columns = ["gameid","gamedate"]
df0["gameid"] = df0["gameid"].astype(int)
df1 = get_box("T","Adv",False,[2024])
df1 = pd.merge(df0,df1,on="gameid")
df1 = df1[['gameid', 'gamedate', 'teamid','teamtricode',
       'offensiverating', 'defensiverating', 'netrating', 
       'pace', 'possessions']]
df1["pts11"] = df1["offensiverating"] * df1["possessions"]
df1["pts12"] = df1["defensiverating"] * df1["possessions"]

In [ ]:
df2 = df1.groupby(["teamid","teamtricode"])[["possessions","pts11","pts12"]].sum()
df2 = df2.reset_index()
df2["offensiverating"] = round(df2["pts11"]/df2["possessions"],1)
df2["defensiverating"] = round(df2["pts12"]/df2["possessions"],1)
df2["netrating"] = round(df2["offensiverating"] - df2["defensiverating"],1)
df2 = df2.drop(columns=["possessions","pts11","pts12"])

In [ ]:
df3 = df1.sort_values(by=["teamtricode","gamedate"]).reset_index(drop=True)
df3["dated"] = (df3["gamedate"] - df3["gamedate"].iloc[-1]).dt.days
teams = df3["teamtricode"].unique()
scale = 1e-2
df3["tdecay"] = round(np.exp(scale*df3["dated"]),4)
df3["pts21"] = df3["pts11"]*df3["tdecay"]
df3["pts22"] = df3["pts12"]*df3["tdecay"]
df3["possessions2"] = df3["possessions"]*df3["tdecay"]

In [ ]:
df3[["gamedate","teamtricode","dated","tdecay"]]

In [ ]:
df4 = df3.groupby(["teamid","teamtricode"])[["possessions2","pts21","pts22"]].sum()
df4 = df4.reset_index()
df4["offensiverating"] = round(df4["pts21"]/df4["possessions2"],1)
df4["defensiverating"] = round(df4["pts22"]/df4["possessions2"],1)
df4["netrating"] = round(df4["offensiverating"] - df4["defensiverating"],1)
df4 = df4.drop(columns=["possessions2","pts21","pts22"])

In [ ]:
df5 = pd.merge(df2,df4,on=["teamid","teamtricode"],suffixes=["","_t"])
df5["netrating_d"] = df5["netrating_t"] - df5["netrating"]
df5["team"] = df5["teamid"].map(team_dict)
df5["image"] = img_DIR_T + df5["team"] + ".png"

In [ ]:
p = (
    ggplot(df5)
    + aes(x="netrating",y="netrating_d",image="image")
    + geom_image()
    + geom_hline(yintercept=0, linetype="dashed", color="red", size=0.5)
    + geom_vline(xintercept=0, linetype="dashed", color="red", size=0.5)
    + theme_idv
    + theme(
        figure_size=(8,8),
        plot_title=element_text(weight="bold",size=24),
        plot_subtitle=element_text(size=14),
    )
    + pnba
    + labs(
        x="Net Rating",
        y="Hotness",
        title="Team Hotness vs Net Rating",
        subtitle="2024-25 NBA Season, as of "+ datetime.today().strftime("%Y-%m-%d")+"\n"+"Hotness is the difference between Time-decay Net Rating and overall Net Rating"
    )
    
)
p

## Saturated Net Rating

In [ ]:
df1 = get_box("T","Adv",False,[2024])
df1 = df1[['gameid', 'teamid','teamtricode',
       'offensiverating', 'defensiverating', 'netrating', 
       'pace', 'possessions']]

In [ ]:
df1["netrating_s"] = df1["netrating"]
df1.loc[df1["netrating"]>20,"netrating_s"] = 20
df1.loc[df1["netrating"]<-20,"netrating_s"] = -20

In [ ]:
df1["net1p"] = df1["netrating"]*df1["possessions"]
df1["net2p"] = df1["netrating_s"]*df1["possessions"]

In [ ]:
df2 = df1.groupby(["teamid","teamtricode"])[["possessions","net1p","net2p"]].sum()
df2 = df2.reset_index()
df2["netrating"] = round(df2["net1p"]/df2["possessions"],1)
df2["netrating_s"] = round(df2["net2p"]/df2["possessions"],1)
df2["netrating_d"] = df2["netrating_s"] - df2["netrating"]
df2["team"] = df2["teamid"].map(team_dict)
df2["image"] = img_DIR_T + df2["team"] + ".png"

In [ ]:
today = datetime.today().strftime("%Y-%m-%d")
subtitle = ("2024-25 NBA Season, as of " +  r"$\bf{" + today + "}$" + "\n"+
r"$\bf{" +"Blowout\,Factor"+"}$ is the difference between Saturated Net Rating and overall Net Rating"+ "\n"+
r"$\bf{" + "Saturated\,Net\,Rating:" + "}$ Limiting maximum and minimum net rating at 20 and -20")

p = (
    ggplot(df2)
    + aes(x="netrating_s",y="netrating_d",image="image")
    + geom_image()
    + geom_hline(yintercept=0, linetype="dashed", color="red", size=0.5)
    + geom_vline(xintercept=0, linetype="dashed", color="red", size=0.5)
    + theme_idv
    + theme(
        figure_size=(8,8),
        plot_title=element_text(weight="bold",size=22),
        plot_subtitle=element_text(size=12),
    )
    + pnba
    + labs(
        x="Saturated Net Rating",
        y="Blowout Factor",
        title="Team Blowout Factor vs Saturated Net Rating",
        subtitle=subtitle
    )
)
p

## Saturated Net Rating v2

In [ ]:
df1 = get_box("T","Adv",False,[2024])
df1 = df1[['gameid', 'teamid','teamtricode',
       'offensiverating', 'defensiverating', 'netrating', 
       'pace', 'possessions']]

In [ ]:
pow = 0.8
df1["netrating_s"] = np.sign(df1["netrating"])*abs(df1["netrating"])**pow
# df1.loc[df1["netrating"]>20,"netrating_s"] = 20
# df1.loc[df1["netrating"]<-20,"netrating_s"] = -20

In [ ]:
df1[["netrating","netrating_s"]].query("netrating>20")

In [ ]:
df1["net1p"] = df1["netrating"]*df1["possessions"]
df1["net2p"] = df1["netrating_s"]*df1["possessions"]

In [ ]:
df2 = df1.groupby(["teamid","teamtricode"])[["possessions","net1p","net2p"]].sum()
df2 = df2.reset_index()
df2["netrating"] = round(df2["net1p"]/df2["possessions"],1)
df2["netrating_s"] = round(df2["net2p"]/df2["possessions"],1)
df2["netrating_d"] = df2["netrating_s"] - df2["netrating"]
df2["team"] = df2["teamid"].map(team_dict)
df2["image"] = img_DIR_T + df2["team"] + ".png"

In [ ]:
today = datetime.today().strftime("%Y-%m-%d")
subtitle = ("2024-25 NBA Season, as of " +  r"$\bf{" + today + "}$" + "\n"+
r"$\bf{" +"Blowout\,Factor"+"}$ is the difference between Saturated Net Rating and overall Net Rating"+ "\n"+
r"$\bf{" + "Saturated\,Net\,Rating:" + "}$ Limiting maximum and minimum net rating at 20 and -20")

p = (
    ggplot(df2)
    + aes(x="netrating_s",y="netrating_d",image="image")
    + geom_image()
    + geom_hline(yintercept=0, linetype="dashed", color="red", size=0.5)
    + geom_vline(xintercept=0, linetype="dashed", color="red", size=0.5)
    + theme_idv
    + theme(
        figure_size=(8,8),
        plot_title=element_text(weight="bold",size=22),
        plot_subtitle=element_text(size=12),
    )
    + pnba
    + labs(
        x="Saturated Net Rating",
        y="Blowout Factor",
        title="Team Blowout Factor vs Saturated Net Rating",
        subtitle=subtitle
    )
)
p